In [492]:
import pandas as pd

In [493]:
# pip install ludwig[full]

In [494]:
url = 'https://raw.githubusercontent.com/john-adeojo/walmartdata/main/Walmart%20Store/TRAIN.csv'
df = pd.read_csv(url)
df.head()

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52


In [495]:
df['Date'] = pd.to_datetime(df['Date'])

In [496]:
# Data set splitting
import numpy as np
import hashlib

def split_data(df):
  # Create a new column 'hash_val' that is the hash of the 'Store_id' column
  df['hash_val'] = df['Store_id'].apply(lambda x: int(hashlib.sha256(str(x).encode('utf-8')).hexdigest(), 16))

  # Use the 'hash_val' column to create a boolean mask for the holdout set
  is_holdout = df['hash_val'] % 10 < 2  # Approximately 20% will be in the holdout set

  # Create the holdout and train sets
  holdout_set = df[is_holdout].copy()
  train_set = df[~is_holdout].copy()

  # Add a 'set' column to each set
  holdout_set['set'] = 'hold_out'
  train_set['set'] = 'train'

  df_predictions = pd.concat([holdout_set, train_set], axis=0)

  return train_set, df_predictions

train_set, df_predictions = split_data(df)

In [497]:
import pandas as pd
import numpy as np

df = train_set

def transform_data(df):

  # Convert 'Discount' column to binary
  df['Discount'] = df['Discount'].map({'Yes': 1, 'No': 0}).astype(int)

  # Convert 'Date' to datetime
  df['Date'] = pd.to_datetime(df['Date'])

  # Create 'DayOfWeek' and 'MonthOfYear'
  df['DayOfWeek'] = df['Date'].dt.dayofweek
  df['MonthOfYear'] = df['Date'].dt.month

  # Sort DataFrame by 'Store_id' and 'Date'
  df.sort_values(['Store_id', 'Date'], inplace=True)

  # List of sequence features
  sequence_features = ['Sales', '#Order', 'Discount', 'DayOfWeek', 'MonthOfYear', 'Holiday']

  # Window size for features and labels (3 days for example)
  feature_window_size = 210
  label_window_size = 30

  # List to store sequences
  sequences = []

  # Generate sequences for each store
  for store_id in df['Store_id'].unique():
      df_store = df[df['Store_id'] == store_id]

      # Check if store has enough data for the window
      if len(df_store) >= (feature_window_size + label_window_size):
          sequence = {feature: ' '.join(map(str, df_store[feature].iloc[-feature_window_size-label_window_size:-label_window_size].values)) for feature in sequence_features}
          sequence['Sales_sequence_label'] = ' '.join(map(str, df_store['Sales'].iloc[-label_window_size:].values))
          sequence['Sales_sequence_label_date'] = ' '.join(map(str, df_store['Date'].iloc[-label_window_size:].dt.date.values))
          sequence['Store_id'] = store_id
          sequences.append(sequence)

  # Convert list of sequences to DataFrame
  df_sequences = pd.DataFrame(sequences)

  df_sequences.rename(columns={'#Order': 'Order'}, inplace=True)

  # Split 'Sales_sequence_label' and 'Sales_sequence_label_date' into list of values
  df_sequences['Sales_sequence_label'] = df_sequences['Sales_sequence_label'].str.split(' ')
  df_sequences['Sales_sequence_label_date'] = df_sequences['Sales_sequence_label_date'].str.split(' ')

  # Determine the maximum length of sales sequences
  max_length = df_sequences['Sales_sequence_label'].str.len().max()

  # Convert list into separate columns
  sales_columns = df_sequences['Sales_sequence_label'].apply(pd.Series)
  sales_columns_date = df_sequences['Sales_sequence_label_date'].apply(pd.Series)

  # Rename columns
  sales_columns = sales_columns.rename(columns = lambda x : 'Sales_sequence_label_' + str(df_sequences['Sales_sequence_label_date'].iloc[0][x]))

  # Concatenate the sales_columns dataframe with the original dataframe
  df_sequences = pd.concat([df_sequences[:], sales_columns[:]], axis=1)

  # Drop the original 'Sales_sequence_label' and 'Sales_sequence_label_date' columns
  df_sequences = df_sequences.drop(['Sales_sequence_label', 'Sales_sequence_label_date'], axis=1)

  return df_sequences

df_sequences_train = transform_data(train_set)
df_sequences_all = transform_data(df_predictions)
df_sequences_train

,Sales,Order,Discount,DayOfWeek,MonthOfYear,Holiday,Store_id,Sales_sequence_label_2019-05-02,Sales_sequence_label_2019-05-03,Sales_sequence_label_2019-05-04,...,Sales_sequence_label_2019-05-22,Sales_sequence_label_2019-05-23,Sales_sequence_label_2019-05-24,Sales_sequence_label_2019-05-25,Sales_sequence_label_2019-05-26,Sales_sequence_label_2019-05-27,Sales_sequence_label_2019-05-28,Sales_sequence_label_2019-05-29,Sales_sequence_label_2019-05-30,Sales_sequence_label_2019-05-31
0,38616.0 48669.0 23844.0 34293.0 26871.0 36351....,64 80 43 63 52 69 46 53 34 68 71 85 80 46 49 2...,1 1 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 1 1 ...,3 4 5 6 0 1 2 3 4 5 6 0 1 2 3 4 5 6 0 1 2 3 4 ...,10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 1...,0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 0 ...,1,34710.0,52362.0,66564.0,...,36753.0,41790.0,32700.0,40554.0,25035.0,33075.0,37317.0,44652.0,42387.0,39843.78
1,68748.0 71817.0 56346.0 80508.0 62217.0 64317....,133 135 112 159 127 129 138 136 86 128 130 117...,1 1 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 1 1 ...,3 4 5 6 0 1 2 3 4 5 6 0 1 2 3 4 5 6 0 1 2 3 4 ...,10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 1...,0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 0 ...,3,106422.0,131055.0,143373.0,...,87942.0,76677.0,76899.0,94548.0,66036.0,69930.0,72540.0,76428.0,78135.0,75790.95000000001
2,22845.0 33576.0 35820.0 38391.0 26811.0 34671....,41 61 67 71 47 64 42 65 67 81 87 83 83 29 41 3...,0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 ...,3 4 5 6 0 1 2 3 4 5 6 0 1 2 3 4 5 6 0 1 2 3 4 ...,10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 1...,0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 0 ...,4,56964.0,59370.0,80760.0,...,40182.0,26343.0,42471.0,49185.0,53550.0,48219.0,55194.0,25938.0,37119.0,36747.81
3,45021.0 56634.0 47181.0 52395.0 38658.0 36105....,71 88 82 86 67 60 64 81 64 84 78 66 63 56 53 4...,1 1 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 1 1 ...,3 4 5 6 0 1 2 3 4 5 6 0 1 2 3 4 5 6 0 1 2 3 4 ...,10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 1...,0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 0 ...,5,52962.0,64221.0,71742.0,...,50922.0,41766.0,38847.0,55830.0,45726.0,48849.0,46806.0,43197.0,46737.0,44867.52
4,53763.0 20844.0 52314.0 43356.0 40845.0 34440....,96 40 102 83 79 67 72 34 85 99 99 103 97 81 26...,1 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 ...,3 4 5 6 0 1 2 3 4 5 6 0 1 2 3 4 5 6 0 1 2 3 4 ...,10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 1...,0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 0 ...,6,77808.0,77703.0,103794.0,...,44355.0,49653.0,47787.0,27150.0,60165.0,52029.0,53100.0,67809.0,72012.0,67691.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,40629.0 42645.0 47724.0 48795.0 38718.0 49641....,71 73 77 85 66 82 94 66 68 74 78 69 66 55 51 4...,0 0 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 ...,3 4 5 6 0 1 2 3 4 5 6 0 1 2 3 4 5 6 0 1 2 3 4 ...,10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 1...,0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 0 ...,358,42771.0,38673.0,45927.0,...,39138.0,44979.0,47772.0,57072.0,59562.0,49035.0,55506.0,59139.0,44934.0,44934.0
288,39885.0 25095.0 27093.0 24495.0 19227.0 19023....,53 38 39 37 29 29 31 37 41 43 41 38 37 35 26 1...,1 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 ...,3 4 5 6 0 1 2 3 4 5 6 0 1 2 3 4 5 6 0 1 2 3 4 ...,10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 1...,0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 0 ...,359,42654.0,42420.0,52806.0,...,25221.0,27714.0,25935.0,34569.0,33342.0,29385.0,33381.0,34797.0,26583.0,24988.02
289,32064.0 31854.0 29307.0 28539.0 19296.0 23943....,55 56 53 52 36 45 45 53 57 68 75 82 77 39 33 3...,0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 ...,3 4 5 6 0 1 2 3 4 5 6 0 1 2 3 4 5 6 0 1 2 3 4 ...,10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 1...,0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 0 ...,361,45756.0,54618.0,83799.0,...,24102.0,29337.0,30393.0,33981.0,34881.0,32715.0,46212.0,24372.0,25662.0,25405.38
290,36762.0 47310.0 26472.0 39138.0 36120.0 33726....,65 80 47 69 69 64 70 85 35 74 68 53 52 45 47 2...

In [498]:
import requests
import yaml
from ludwig.api import LudwigModel

# URL of the raw YAML file in the GitHub repository
url = 'https://raw.githubusercontent.com/john-adeojo/walmartdata/main/Walmart%20Store/timerseries_v3.yaml'

# Send a GET request to the URL
response = requests.get(url)

# Raise an exception if the request was unsuccessful
response.raise_for_status()

# Load the YAML data from the response text
config = yaml.safe_load(response.text)

# Now you can use the config dictionary to initialize the Ludwig model
model = LudwigModel(config=config)
results = model.train(dataset=df_sequences_train)

In [499]:
predictions, _ = model.predict(dataset=df_sequences_all)
predictions.head()

,Sales_sequence_label_2019-05-02_predictions,Sales_sequence_label_2019-05-03_predictions,Sales_sequence_label_2019-05-04_predictions,Sales_sequence_label_2019-05-05_predictions,Sales_sequence_label_2019-05-06_predictions,Sales_sequence_label_2019-05-07_predictions,Sales_sequence_label_2019-05-08_predictions,Sales_sequence_label_2019-05-09_predictions,Sales_sequence_label_2019-05-10_predictions,Sales_sequence_label_2019-05-11_predictions,...,Sales_sequence_label_2019-05-22_predictions,Sales_sequence_label_2019-05-23_predictions,Sales_sequence_label_2019-05-24_predictions,Sales_sequence_label_2019-05-25_predictions,Sales_sequence_label_2019-05-26_predictions,Sales_sequence_label_2019-05-27_predictions,Sales_sequence_label_2019-05-28_predictions,Sales_sequence_label_2019-05-29_predictions,Sales_sequence_label_2019-05-30_predictions,Sales_sequence_label_2019-05-31_predictions
0,36733.433594,37906.925781,44493.136719,33357.734375,28171.318359,27677.509766,25139.888672,23032.957031,29463.935547,33742.390625,...,27347.763672,27318.076172,26352.447266,33988.554688,35350.765625,30736.341797,32591.001953,31583.074219,28154.230469,27254.701172
1,71427.492188,72096.375000,84372.382812,60203.000000,49877.195312,48888.679688,48397.058594,48375.785156,50998.796875,59638.660156,...,46065.136719,46585.468750,48017.324219,60745.875000,63570.734375,53225.261719,56729.234375,53789.906250,48856.925781,47248.996094
2,84140.250000,86484.632812,100637.234375,74182.781250,62622.367188,60227.683594,56694.566406,54972.933594,62926.531250,72866.976562,...,56619.476562,56825.265625,55491.453125,72270.406250,76520.578125,65225.035156,69766.703125,65557.546875,59450.605469,57653.332031
3,57716.664062,58136.703125,68177.039062,48501.820312,40076.113281,39561.113281,39219.902344,38991.714844,41427.125000,48394.718750,...,37467.636719,38097.406250,39525.160156,49641.843750,51685.886719,43321.062500,46088.804688,44028.730469,39907.972656,38563.996094
4,51752.984375,52613.847656,61622.156250,44619.503906,37212.523438,36542.996094,35157.917969,34154.058594,38443.289062,44589.820312,...,34827.675781,35325.183594,35724.781250,45306.664062,47275.609375,40105.886719,42684.835938,40838.132812,36892.812500,35683.699219


In [500]:
# First, let's reset the index of predictions dataframe to have the store id as a column
predictions = predictions.reset_index().rename(columns={'index': 'Store_id'})

# Now we convert the wide format dataframe to a long format dataframe
predictions_melted = predictions.melt(id_vars=['Store_id'], var_name='Date', value_name='Predicted_Sales')

# The 'Date' column is currently a string in the format 'Sales_sequence_label_YYYY-MM-DD_predictions', let's extract the date
predictions_melted['Date'] = predictions_melted['Date'].str.extract('(\d{4}-\d{2}-\d{2})')

# Convert 'Date' column back to datetime format
predictions_melted['Date'] = pd.to_datetime(predictions_melted['Date'])

# Now, let's merge this with the original dataframe
df_analysis = pd.merge(df_predictions, predictions_melted, on=['Store_id', 'Date'], how='left')

In [501]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Function to calculate RMSE
def calculate_rmse(group):
    actual = group['Sales']
    predicted = group['Predicted_Sales']
    rmse = sqrt(mean_squared_error(actual, predicted))
    return rmse

# Apply the function to each group
rmse_by_set = df_analysis.loc[df_analysis['Predicted_Sales'].notnull()].groupby('set').apply(calculate_rmse)

print(rmse_by_set)

set
hold_out    26414.337897
train       26347.921072
dtype: float64


In [502]:
import plotly.express as px

plot_data = df_analysis.loc[df_analysis['Predicted_Sales'].notnull()]

samples = plot_data.loc[plot_data['set'] == 'hold_out']
samples = samples.sample(n=10)
samples_list = list(samples['Store_id'])

for store in samples_list:
  # Select the data for a specific store
  store_id = store  # replace with your store id
  df_store = plot_data[plot_data['Store_id'] == store_id]

  # Create a line plot of sales over time
  fig = px.line(df_store, x='Date', y='Sales', title='Sales Over Time for Store {}'.format(store_id))

  # Add a line for predicted sales
  fig.add_trace(go.Scatter(x=df_store['Date'], y=df_store['Predicted_Sales'], mode='lines', name='Predicted Sales'))

  fig.show()

In [503]:
plot_data['error'] = plot_data['Sales'] - plot_data['Predicted_Sales']

<ipython-input-503-cded490abcde>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [504]:
import plotly.express as px

# Assuming 'category' is the name of your category column
fig = px.histogram(plot_data, x="error", color="set", nbins=30,
                   labels={"error": "Error"},
                   title="Histogram of Error by Category",
                   template='plotly_white')

fig.show()
